In [ ]:
# data, metadata = csp.io.from_fca_repo('newzealand_en')
# data

In [ ]:
# import caspailleur as csp

In [ ]:
# objects = set(data.index)
# attributes = set(data.columns)
# connections = {(obj, attr) for obj, attrs in data.iterrows() for attr in attrs[attrs].index}

In [3]:
objects = {
    'Catlins', 'Dunedin', 'Fjordland NP', 'Haast', 'Invercargill',
    'MT. Aspiring NP', 'Milford Sound', 'Oamaru', 'Otago Peninsula','Queenstown',
    'Stewart Island', 'Te Anau', 'Wanaka'
}
attributes = {
    'Bungee Jumping', 'Hiking', 'Jet Boating', 'Observing Nature', 'Parachute Gliding',
    'Sightseeing Flights', 'Skiing', 'Wildwater Rafting'
}
connections = {
    ('Catlins', 'Hiking'), ('Catlins', 'Observing Nature'),
    ('Dunedin', 'Hiking'), ('Dunedin', 'Observing Nature'), ('Dunedin', 'Sightseeing Flights'),
    ('Fjordland NP', 'Hiking'), ('Fjordland NP', 'Observing Nature'), ('Fjordland NP', 'Sightseeing Flights'),
    ('Haast', 'Hiking'), ('Haast', 'Observing Nature'),
    ('Invercargill', 'Hiking'), ('Invercargill', 'Observing Nature'), ('Invercargill', 'Sightseeing Flights'),
    ('MT. Aspiring NP', 'Hiking'), ('MT. Aspiring NP', 'Observing Nature'), ('MT. Aspiring NP', 'Sightseeing Flights'),
    ('Milford Sound', 'Hiking'), ('Milford Sound', 'Observing Nature'), ('Milford Sound', 'Sightseeing Flights'),
    ('Oamaru', 'Hiking'), ('Oamaru', 'Observing Nature'),
    ('Otago Peninsula', 'Hiking'), ('Otago Peninsula', 'Observing Nature'),
    ('Queenstown', 'Bungee Jumping'), ('Queenstown', 'Hiking'), ('Queenstown', 'Jet Boating'), ('Queenstown', 'Parachute Gliding'),
    ('Queenstown', 'Sightseeing Flights'), ('Queenstown', 'Skiing'), ('Queenstown', 'Wildwater Rafting'),
    ('Stewart Island', 'Hiking'), ('Stewart Island', 'Observing Nature'), ('Stewart Island', 'Sightseeing Flights'),
    ('Te Anau', 'Hiking'), ('Te Anau', 'Jet Boating'), ('Te Anau', 'Observing Nature'), ('Te Anau', 'Sightseeing Flights'),
    ('Wanaka', 'Bungee Jumping'), ('Wanaka', 'Hiking'), ('Wanaka', 'Jet Boating'), ('Wanaka', 'Parachute Gliding'),
    ('Wanaka', 'Sightseeing Flights'),('Wanaka', 'Skiing'), ('Wanaka', 'Wildwater Rafting')
}
context = (objects, attributes, connections)

# Part 0. Basic functions

Mathematical definition: $$\mathtt{ext}(B) = B' = \{g \in G \mid (g, m) \in I,\  \forall m \in B\}$$

In [25]:
def extent(description: set[str], context: tuple[set, set, set]) -> set[str]:
    # TODO: Implement the function
    G, M, I = context
    ext =  {obj for obj in G if all((obj, attr) in I for attr in description)}
    return ext

In [26]:
assert extent({'Bungee Jumping'}, context) == {'Queenstown', 'Wanaka'}
assert extent({'Hiking', 'Observing Nature'}, context) == extent({'Observing Nature'}, context)

Mathematical definition: $$\mathtt{int}(A) = A' = \{m \in M \mid (g, m) \in I,\ \forall g \in A\}$$

In [27]:
def intent(some_objects: set[str], context: tuple[set, set, set]) -> set[str]:
    # TODO: Implement the function
    G, M, I = context
    intent  = {attr for attr in M if all((obj, attr) in I for obj in some_objects)}
    return intent

In [28]:
assert intent({"Stewart Island"}, context) == {'Hiking', 'Observing Nature', 'Sightseeing Flights'}
assert intent({'Fjordland NP', 'Stewart Island'}, context) == intent({'Stewart Island'}, context)

Mathematical definition: $$\mathtt{supp}(B) = |B'|$$

In [29]:
def support(description: set[str], context: tuple[set, set, set]) -> int:
    # TODO: Implement the function
    sup = len(extent(description, context))
    return sup

In [30]:
assert support({'Hiking'}, context) == 13
assert support({'Sightseeing Flights', 'Parachute Gliding'}, context) == 2

Applying implications to a description

In [33]:
# Apply implications to a description in one go (assuming that implications are specifically ordered)
def saturate_direct(description: set[str], direct_implications: list[tuple[set[str, set[str]]]]) -> set[str]:
  # TODO: Implement the function
  saturated = set(description)
  for premise, conclusion in direct_implications:
    if premise <= saturated:
      saturated |= conclusion

  return saturated

In [34]:
implications = [
    (set(), {'Hiking'}),
    ({'Parachute Gliding', "Hiking"}, {'Sightseeing Flights'})
]
assert saturate_direct({'Parachute Gliding'}, implications) == {'Parachute Gliding', 'Hiking', 'Sightseeing Flights'}
assert not saturate_direct({'Parachute Gliding'}, implications[::-1]) == saturate_direct({'Parachute Gliding'}, implications)

In [35]:
# APply implications to a description in a trustfull bruteforce way
def saturate(
    description: set[str],
    implications: list[tuple[set[str], set[str]]],
):
  # TODO: Implement the function
  saturated, saturated_old = set(description), set()
  while saturated != saturated_old:
    saturated_old = set(saturated)
    for premise, conclusion in implications:
      if premise <= saturated:
        saturated |= conclusion
  return saturated

In [36]:
implications = [(set(), {'Hiking'}), ({'Parachute Gliding'}, {'Sightseeing Flights'})]
assert saturate({'Parachute Gliding'}, implications) == {'Parachute Gliding', 'Hiking', 'Sightseeing Flights'}
assert saturate({'Parachute Gliding'}, implications[::-1]) == saturate({'Parachute Gliding'}, implications)

# Part 1. Bruteforce traversal

## 1.1 Generic Breadth-first traversal

In [ ]:
from datetime import datetime

class DescriptionTraverser:
    def __init__(self):
        self.passed_descriptions = None
        self.traversal_time = None
        self.goal_descriptions = None

    def traverse(self, K: tuple[set, set, set]):
        self.preprocess_context(K)

        self.passed_descriptions = []

        time_start = datetime.now()
        queue: list[set[str]] = [set()]
        while queue:
            description = queue.pop(0)
            self.passed_descriptions.append(description)

            if self.abort_description(description, K):
                continue

            if self.is_goal_description(description, K):
                self.register_goal_description(description)

            queue.extend(self.sucessors(description, K))
        time_end = datetime.now()
        self.traversal_time = (time_end-time_start).total_seconds()

    def abort_description(self, description: set[str], K: tuple[set, set, set]) -> bool:
        return False

    def is_goal_description(self, description: set[str], K: tuple[set, set, set]) -> bool:
        return False

    def register_goal_description(self, description: set[str]):
        if self.goal_descriptions is None:
            self.goal_descriptions = set()
        self.goal_descriptions.add(frozenset(description))

    def sucessors(self, description: set[str], K: tuple[set, set, set]) -> list[set[str]]:
        G, M, I = K
        return [description|{m} for m in M-description]

    def preprocess_context(self, K: tuple[set, set, set]) -> tuple[set, set, set]:
        G, M, I = K
        self.attributes_order = sorted(M)  # from index to attribute
        self.attributes_index_dict = {m: idx for idx, m in enumerate(self.attributes_order)}  # from attribute to index

    def print_stats(self, list_all_goals: bool = True):
        print(f"{traverser.traversal_time=} sec.")
        print(f"{len(traverser.passed_descriptions)=}")
        goals = self.goal_descriptions if self.goal_descriptions is not None else []
        print(f"{len(goals)=}")
        if list_all_goals:
            if goals:
                goals = sorted(map(set, traverser.goal_descriptions), key=lambda goal: (len(goal), goal))
                print("goals:", *goals, sep='\n * ')
            else:
                print("goals:", "None")

In [ ]:
traverser = DescriptionTraverser()
traverser.traverse(context)

traverser.print_stats()

traverser.traversal_time=1.180899 sec.
len(traverser.passed_descriptions)=109601
len(goals)=0
goals: None


## 1.2 Bruteforce search for min.generators, intents, etc

### Intent (aka maximal description, closed description)

Mathematical definition:
$$D \text{ is intent} \iff D'' = \mathtt{int}(\mathtt{ext}(D)) = D$$

In [ ]:
def is_intent(description: set[str], context: tuple[set, set, set]) -> bool:
    # TODO: Implement the function
    return ...

In [ ]:
traverser = DescriptionTraverser()
traverser.is_goal_description = is_intent

traverser.traverse(context)
assert len(traverser.goal_descriptions) == 8

traverser.print_stats()

traverser.traversal_time=2.35725 sec.
len(traverser.passed_descriptions)=109601
len(goals)=8
goals:
 * {'Hiking'}
 * {'Observing Nature', 'Hiking'}
 * {'Sightseeing Flights', 'Hiking'}
 * {'Sightseeing Flights', 'Jet Boating', 'Hiking'}
 * {'Sightseeing Flights', 'Observing Nature', 'Hiking'}
 * {'Sightseeing Flights', 'Observing Nature', 'Jet Boating', 'Hiking'}
 * {'Parachute Gliding', 'Skiing', 'Hiking', 'Sightseeing Flights', 'Wildwater Rafting', 'Bungee Jumping', 'Jet Boating'}
 * {'Parachute Gliding', 'Hiking', 'Sightseeing Flights', 'Bungee Jumping', 'Skiing', 'Observing Nature', 'Wildwater Rafting', 'Jet Boating'}


### Min.generator

Mathematical definition: $$D \text{ is min. gen.} \iff \forall m \in D,\ D'' \neq (D \setminus \{m\})''$$

In [ ]:
def is_mingen(description: set[str], context: tuple[set, set, set]) -> bool:
    # TODO: Implement the function
    return ...

In [ ]:
traverser = DescriptionTraverser()
traverser.is_goal_description = is_mingen
traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14

traverser.print_stats()

NameError: name 'DescriptionTraverser' is not defined

### Proper premise

Mathematical definition:
$$D \text{ is proper premise} \iff D \cup \bigcup\limits_{m \in D} (D \setminus {m})'' \neq D''$$

In [ ]:
def is_proper_premise(description: set[str], context: tuple[set, set, set]) -> bool:
    # TODO: Implement the function
    return ...

In [ ]:
traverser = DescriptionTraverser()
traverser.is_goal_description = is_proper_premise
traverser.traverse(context)

assert len(traverser.goal_descriptions) == 6

traverser.print_stats()

traverser.traversal_time=15.428917 sec.
len(traverser.passed_descriptions)=109601
len(goals)=6
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Jet Boating'}


## Pseudo-intent*

Mathematical definition:
$$D \text{ is pseudo-intent} \iff D'' \neq D \text{ and } P'' \subset D, \forall P \in \text{PseudoIntents}, \text{ s.t. } P \subset D$$

In [ ]:
def is_pseudo_intent(description: set[str], context: tuple[set, set, set], other_pseudo_intents: set[set[str]]) -> bool:
    # TODO: Implement the function
    return ...

class PIntentTraverser(DescriptionTraverser):
    def is_goal_description(self, description, K) -> bool:
        goals = self.goal_descriptions if self.goal_descriptions is not None else []
        return is_pseudo_intent(description, K, goals)

NameError: name 'DescriptionTraverser' is not defined

In [ ]:
traverser = PIntentTraverser()
traverser.traverse(context)

assert len(traverser.goal_descriptions) == 6

traverser.print_stats()

traverser.traversal_time=4.382236 sec.
len(traverser.passed_descriptions)=109601
len(goals)=6
goals:
 * set()
 * {'Jet Boating', 'Hiking'}
 * {'Bungee Jumping', 'Hiking'}
 * {'Wildwater Rafting', 'Hiking'}
 * {'Parachute Gliding', 'Hiking'}
 * {'Skiing', 'Hiking'}


# Part 2. Optimise the traversal

## 2.1 Cut the search space

In [ ]:
def is_big_or_rare(description: set[str], K: tuple[set, set, set], min_support: int = 1, max_length: int = 3) -> bool:
    # TODO: Implement the function
    return ...

In [ ]:
traverser = DescriptionTraverser()
traverser.is_goal_description = is_mingen
traverser.abort_description = is_big_or_rare
traverser.traverse(context)

assert len(traverser.goal_descriptions) == 10

traverser.print_stats()

traverser.traversal_time=0.004037 sec.
len(traverser.passed_descriptions)=353
len(goals)=10
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


## 2.2 Smart Search space navigation

### 2.2.1. Do not step the same node twice

In [ ]:
# A solution that is used when traversing a generic graph
class RememberingTraverser(DescriptionTraverser):
  def abort_description(self, description: set[str], K) -> bool:
    return description in self.passed_descriptions[:-1]

In [ ]:
# A solution that is used when traversing a graph where are subsets of some set
class LexicTraverser(DescriptionTraverser):
    def preprocess_context(self, K: tuple[set, set, set]) -> tuple[set, set, set]:
        G, M, I = K
        self.attributes_order = sorted(M)  # from index to attribute
        self.attributes_index_dict = {m: idx for idx, m in enumerate(self.attributes_order)}  # from attribute to index
        return K

    def sucessors(self, description: set[str], K: tuple[set, set, set]) -> list[set[str]]:
        # TODO: Implement the function
        return ...

NameError: name 'DescriptionTraverser' is not defined

In [ ]:
traverser = DescriptionTraverser()
traverser.is_goal_description = is_mingen

traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14
traverser.print_stats()

traverser.traversal_time=14.812431 sec.
len(traverser.passed_descriptions)=109601
len(goals)=14
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Bungee Jumping', 'Observing Nature'}
 * {'Parachute Gliding', 'Observing Nature'}
 * {'Skiing', 'Observing Nature'}
 * {'Wildwater Rafting', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


In [ ]:
traverser = RememberingTraverser()
traverser.is_goal_description = is_mingen

traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14
traverser.print_stats()

traverser.traversal_time=0.036039 sec.
len(traverser.passed_descriptions)=1025
len(goals)=14
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Bungee Jumping', 'Observing Nature'}
 * {'Parachute Gliding', 'Observing Nature'}
 * {'Skiing', 'Observing Nature'}
 * {'Wildwater Rafting', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


In [ ]:
traverser = LexicTraverser()
traverser.is_goal_description = is_mingen

traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14

traverser.print_stats()

traverser.traversal_time=0.020727 sec.
len(traverser.passed_descriptions)=256
len(goals)=14
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Bungee Jumping', 'Observing Nature'}
 * {'Parachute Gliding', 'Observing Nature'}
 * {'Skiing', 'Observing Nature'}
 * {'Wildwater Rafting', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


### 2.2.2 Involve math properties

Mathematics says to us that
> every subset of a min. generator is a min. generator.

That means that
> if some subdescription of a description is not minimal then the given description is not minimal too

In [ ]:
class MinimalTraverser(LexicTraverser):
    def abort_description(self, description: set[str], K: tuple[set, set, set]) -> bool:
        # TODO: Implement the function
        return ...


NameError: name 'LexicTraverser' is not defined

In [ ]:
traverser = MinimalTraverser()
traverser.is_goal_description = is_mingen

traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14

traverser.print_stats()

traverser.traversal_time=0.010619 sec.
len(traverser.passed_descriptions)=72
len(goals)=14
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Bungee Jumping', 'Observing Nature'}
 * {'Parachute Gliding', 'Observing Nature'}
 * {'Skiing', 'Observing Nature'}
 * {'Wildwater Rafting', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


## 2.3 Optimise every iteration

In [ ]:
class MinGenTraverser(LexicTraverser):
    def __init__(self):
        super().__init__()
        self.goal_supports = None

    def register_goal_description(self, description: set[str]):
        super().register_goal_description(description)
        if self.goal_supports is None:
          self.goal_supports = dict()
        self.goal_supports[frozenset(description)] = support(description, context)

    def is_goal_description(self, description: set[str], K: tuple[set, set, set]) -> bool:
        return is_mingen(description, K)

    def abort_description(self, description: set[str], K: tuple[set, set, set]) -> bool:
        # implement the function
        # test that all subsets of `descriptions` are among already found keys
        # and test that the support of these keys is higher than the support of `description`
        return ...

NameError: name 'LexicTraverser' is not defined

In [ ]:
traverser = MinGenTraverser()

traverser.traverse(context)

assert len(traverser.goal_descriptions) == 14

traverser.print_stats()

traverser.traversal_time=0.002406 sec.
len(traverser.passed_descriptions)=45
len(goals)=14
goals:
 * set()
 * {'Bungee Jumping'}
 * {'Observing Nature'}
 * {'Parachute Gliding'}
 * {'Skiing'}
 * {'Wildwater Rafting'}
 * {'Sightseeing Flights'}
 * {'Jet Boating'}
 * {'Jet Boating', 'Observing Nature'}
 * {'Bungee Jumping', 'Observing Nature'}
 * {'Parachute Gliding', 'Observing Nature'}
 * {'Skiing', 'Observing Nature'}
 * {'Wildwater Rafting', 'Observing Nature'}
 * {'Sightseeing Flights', 'Observing Nature'}


# Part 3. Putting everything together

Find Minimal generators

In [ ]:
t1 = datetime.now()
traverser = MinGenTraverser()
traverser.traverse(context)
mingens = sorted(traverser.goal_descriptions, key=len)
time_to_mingens = (datetime.now()-t1).total_seconds()
mingens

[frozenset(),
 frozenset({'Bungee Jumping'}),
 frozenset({'Observing Nature'}),
 frozenset({'Parachute Gliding'}),
 frozenset({'Skiing'}),
 frozenset({'Wildwater Rafting'}),
 frozenset({'Sightseeing Flights'}),
 frozenset({'Jet Boating'}),
 frozenset({'Jet Boating', 'Observing Nature'}),
 frozenset({'Bungee Jumping', 'Observing Nature'}),
 frozenset({'Observing Nature', 'Parachute Gliding'}),
 frozenset({'Observing Nature', 'Skiing'}),
 frozenset({'Observing Nature', 'Wildwater Rafting'}),
 frozenset({'Observing Nature', 'Sightseeing Flights'})]

Find intents

In [ ]:
t1 = datetime.now()
intents = {mingen: intent(extent(mingen, context), context) for mingen in mingens}
time_to_intents = time_to_mingens + (datetime.now()-t1).total_seconds()

Consruct ProperPremise implication basis (aka Canonical Direct basis)

In [ ]:
t1 = datetime.now()
implications = [(mingen, intents[mingen]) for mingen in mingens]
saturated_premises = {
    premise: saturate_direct(premise, [(p, c) for p, c in implications if p!=premise])
    for premise, _ in implications
}
proper_premise_base = [(premise, conclusion) for premise, conclusion in implications
                       if saturated_premises[premise] != conclusion]
time_to_proper_premise_base = time_to_intents + (datetime.now()-t1).total_seconds()
assert len(proper_premise_base) == 6

Construct PseudoIntent basis (aka Canonical basis)

In [ ]:
t1 = datetime.now()
pseudo_intent_basis = []
for premise, conclusion in proper_premise_base:
  pseudo_intent = saturate(premise, pseudo_intent_basis)
  if pseudo_intent != conclusion:
    pseudo_intent_basis.append((pseudo_intent, conclusion))
time_to_pseudo_intents = time_to_proper_premise_base + (datetime.now()-t1).total_seconds()

assert len(pseudo_intent_basis) == 6

Look at the bases

In [ ]:
print('Proper Premise base:')
for premise, conclusion in proper_premise_base:
  saturated_premise = saturate(premise, [(p, c) for p, c in proper_premise_base if p!=premise])
  print(f"({', '.join(premise)}) => ({', '.join(conclusion - saturated_premise)})")

Proper Premise base:
() => (Hiking)
(Bungee Jumping) => (Parachute Gliding, Sightseeing Flights, Skiing, Wildwater Rafting, Jet Boating)
(Parachute Gliding) => (Sightseeing Flights, Bungee Jumping, Skiing, Wildwater Rafting, Jet Boating)
(Skiing) => (Parachute Gliding, Sightseeing Flights, Bungee Jumping, Wildwater Rafting, Jet Boating)
(Wildwater Rafting) => (Parachute Gliding, Sightseeing Flights, Bungee Jumping, Skiing, Jet Boating)
(Jet Boating) => (Sightseeing Flights)


In [ ]:
print('Pseudo Intent base')
for premise, conclusion in pseudo_intent_basis:
  print(f"({', '.join(premise)}) => ({', '.join(conclusion - premise)})")

Pseudo Intent base
() => (Hiking)
(Bungee Jumping, Hiking) => (Parachute Gliding, Sightseeing Flights, Skiing, Wildwater Rafting, Jet Boating)
(Parachute Gliding, Hiking) => (Sightseeing Flights, Bungee Jumping, Skiing, Wildwater Rafting, Jet Boating)
(Skiing, Hiking) => (Parachute Gliding, Sightseeing Flights, Bungee Jumping, Wildwater Rafting, Jet Boating)
(Wildwater Rafting, Hiking) => (Parachute Gliding, Sightseeing Flights, Bungee Jumping, Skiing, Jet Boating)
(Jet Boating, Hiking) => (Sightseeing Flights)


Final timings:

In [ ]:
print(f"{time_to_mingens=}")
print(f"{time_to_intents=}")
print(f"{time_to_proper_premise_base=}")
print(f"{time_to_pseudo_intents=}")

time_to_mingens=0.001652
time_to_intents=0.002295
time_to_proper_premise_base=0.002782
time_to_pseudo_intents=0.003235
